In [2]:
# DO NOT MODIFY! helper functions and constants
import nbimporter
from p1 import cv2, np, plt, math
from p1 import get_parameters, Gauss2D, filter_image_vec
from p2 import edge_detection_nms

image_list, constants = get_parameters()

#----------------------------------------------------------------------
def visualize(image_name, constants):
    image_rgb = cv2.cvtColor(cv2.imread(image_name), cv2.COLOR_BGR2RGB)
    print("-" * 50 + "\n" + "Original Image:")
    plt.imshow(image_rgb); plt.show() # Displaying the sample image
    
    image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    image_m, image_o, image_x, image_y = edge_detection_nms(image, constants.sigma)
    
    print("-" * 50 + "\n" + "Edge Image:")
    plt.imshow(image_m, cmap="gray"); plt.show() # Displaying the sample image
    
    image_m_thres = 1.0*(image_m > constants.thres) # threshold the edge magnitude image
    print("-" * 50 + "\n" + "Thresholded Edge Image:")
    plt.imshow(image_m_thres, cmap="gray"); plt.show() # Displaying the sample image
    
    #--------------hough transform----------------
    H, rho_arr, theta_arr = hough_transform(image_m, constants.thres, constants.rho_res, constants.theta_res)   
    peak_rho_arr, peak_theta_arr = peak_hough_lines(H, rho_arr, theta_arr, constants.num_lines)
    
    #--------------vis using infinitely long lines----------------------------
    vis_line_len = 1000 # len of line in pixels, big enough to span the image
    vis_image_rgb = np.copy(image_rgb)
    for (rho, theta) in zip(peak_rho_arr, peak_theta_arr):
        x0 = rho*np.cos(theta); y0 = rho*np.sin(theta)
        x1 = int(x0 - vis_line_len*np.sin(theta)); y1 = int(y0 + vis_line_len*np.cos(theta))
        x2 = int(x0 + vis_line_len*np.sin(theta)); y2 = int(y0 - vis_line_len*np.cos(theta)); 
        cv2.line(vis_image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    #---------------------------------------------
    print("-" * 50 + "\n" + "Edge Image:")
    plt.imshow(vis_image_rgb); plt.show() # Displaying the sample image
    
    return

### Q3.1 Hough Transform (15 points)

Write a function that applies the Hough Transform to an edge magnitude image.  

```
  def hough_transform(image_m, thres, rho_res, theta_res):
        return H, rho_arr, theta_arr
```

- **Input**: the edge magnitude image *$image\_m$*, scalar *$thres$* to threshold *$image\_m$* into a binary array, scalars *$rho\_res$* and *$theta\_res$* are the resolution of the Hough Transform accumulator along the $\rho$ and $\theta$ axes respectively.
- **Output**: *$H$* is the Hough Transform accumulator, *$rho\_arr$* and *$theta\_arr$* are the arrays of $\rho$ and $\theta$ values over which *$H$* is computed. 

**Instructions**
- Your code cannot call pre-existing functions for Hough Transform.
- We threshold the edge magnitude image first to identify pixels which likely are an edge
- rho_res is the resolution of the rho space, theta_res is the resolution of the theta space in degrees
- rho_arr goes from -rho_max to rho_max in rho_res steps, including the end points!
- theta_arr goes from 0 to pi in theta_res steps, including the end points! Note: theta_arr is in radians not degrees!
- Refer the slides for Hough Transform for the algorithm

**Grading**
- Your code will be autograded with 3 images. 
- The outputs of the function will be compared with the outputs of a reference implementation with an error threshold of 1e-5.

In [8]:
def hough_transform(image_m, thres, rho_res, theta_res):
    """Compute Hough Transform for the image

    Args:
        image_m: np.array, HxW, edge magnitude image.
        thres: float, scalar to threshold image_m
        rho_res: integer, resolution of rho
        theta_res: integer, resolution of theta in degrees
        
    Returns:
        H: np.array, (num of rhos x num of thetas), hough transform accumulator (rho x theta), NOT theta x rho!
        rho_arr: np.array, dim=num of rhos, quantized rhos
        theta_arr: np.array, dim=num of thetas, quantized thetas
    """
    
    image_m_thres = 1.0*(image_m > thres) # threshold the edge magnitude image
    height, width = image_m_thres.shape # image height and width 
    diagonal_len = np.ceil(np.sqrt(height**2 + width**2)) # image diagonal = rho_max   
    rho_max = float(diagonal_len)
    
    # compute rho_arr, we go from [-rho_max to rho_max] in rho_res steps
    rho_arr = np.linspace(
        start=-rho_max,
        stop=rho_max,
        num=int((2 * rho_max / rho_res) + 1),
    )

    # compute theta_arr, we go from [0, pi] in theta_res steps, NOT [-pi/2, pi/2]!
    # Note theta_res is in degrees but theta_scale should be in radians [0, pi]
    theta_arr = np.linspace(
        start=0,
        stop=np.pi,
        num=int((180 / theta_res) + 1),
    )
    
    H = np.zeros((len(rho_arr), len(theta_arr)), dtype=np.int32)
    
    # find all edge (nonzero) pixel indexes
    y_idxs, x_idxs = image_m_thres.nonzero() 
    
    for x, y in zip(x_idxs, y_idxs):
        for theta_idx, theta in enumerate(theta_arr):
            # compute rho_idx, note, theta is in radians!
            # Hint: compute rho first from theta, round it to nearest rho_prime in rho_arr
            # and then find rho_prime's rho_idx (index of rho_prime in rho_arr, NOT index of rho!)

            # distance = pixel.dot([cos, sin]) took  me a while to get, but it works if you
            # define theta starting from +x to the *perpendicular* line, then you see that
            # (y = -x cos / sin + dist / sin) works out exactly like mx + b, where a 90deg triangle
            # is formed with th y-axis, making b = dist / sin. Then you rearrange that line formula
            # to get dist.

            # TODO: Which X and Y are correct? Axes are unclear
            rho = x * np.cos(theta) + y * np.sin(theta)
            rho_idx = np.argmin(np.abs(rho_arr - rho))
            
            H[rho_idx, theta_idx] += 1
    
    return H, rho_arr, theta_arr


# np.set_printoptions(suppress=True, precision=3)
# image = np.zeros((7, 7))
# image[3, :] = 255
# image[:, 3] = 255
# image_m, image_o, image_x, image_y = edge_detection_nms(image, 0.1)
# print(image_m)
# H, rho_arr, theta_arr = hough_transform(image_m, thres=700, rho_res=2, theta_res=18)
# print(f"H\n{H}")
# print(f"rho_arr\n{rho_arr}")
# print(f"theta_arr\n{theta_arr}")
# output = np.zeros((7, 7))

# empty = np.zeros(image.shape)
# for rho_idx, theta_idx in np.argwhere(H == 8):
#     print(f"Line -- distance: {rho_arr[rho_idx]}, angle: {theta_arr[theta_idx]}")

#     rho = rho_arr[rho_idx]
#     theta = theta_arr[theta_idx]
#     vis_line_len = 100
#     x0 = rho*np.cos(theta); y0 = rho*np.sin(theta)
#     x1 = int(x0 - vis_line_len*np.sin(theta)); y1 = int(y0 + vis_line_len*np.cos(theta))
#     x2 = int(x0 + vis_line_len*np.sin(theta)); y2 = int(y0 - vis_line_len*np.cos(theta)); 
#     cv2.line(empty, (x1, y1), (x2, y2), 125, 1)

# print(empty)

[[   0.       0.     806.381  510.     806.381    0.       0.   ]
 [   0.       0.    1020.       0.    1020.       0.       0.   ]
 [ 806.381 1020.    1081.873  510.    1081.873 1020.     806.381]
 [ 510.       0.     510.       0.     510.       0.     510.   ]
 [ 806.381 1020.    1081.873  510.    1081.873 1020.     806.381]
 [   0.       0.    1020.       0.    1020.       0.       0.   ]
 [   0.       0.     806.381  510.     806.381    0.       0.   ]]
H
[[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 3]
 [0 0 0 0 0 0 0 0 2 6 7]
 [0 0 0 0 0 0 1 5 7 6 6]
 [4 1 0 0 1 2 6 6 6 6 4]
 [6 6 6 6 6 8 6 7 5 1 0]
 [8 6 6 6 6 8 6 2 0 0 0]
 [2 7 7 7 7 2 1 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]]
rho_arr
[-10.  -8.  -6.  -4.  -2.   0.   2.   4.   6.   8.  10.]
theta_arr
[0.    0.314 0.628 0.942 1.257 1.571 1.885 2.199 2.513 2.827 3.142]


### Q3.2 Finding Lines (5 points)
Write a function that outputs the indices of the local maxima in Hough Transform accumulator.

```
  def peak_hough_lines(H, rho_arr, theta_arr, num_lines):
        return peak_rho_arr, peak_theta_arr

```

- **Input**: *$H$* is the accumulator matrix (rhos x thetas), *$rho\_arr$* and *$theta\_arr$* are the quantized rho and theta arrays, *$num\_lines$* is a scalar - number of most prominent lines we wish to detect in the image.
- **Output**: *$peak\_rho\_arr$*, *$peak\_theta\_arr$* - top num_lines rhos and thetas.

**Instructions**
- Sort the accumulator H to obtain the top N(num_lines) lines
- Output the rho and theta corresponding to these lines
- We recommend using np.argsort for sorting. This ensures you have consistent tie breaks same as the reference implementation used by autograder.

**Grading** Same as Q3.1

In [7]:
def peak_hough_lines(H, rho_arr, theta_arr, num_lines):
    """Returns the rhos and thetas corresponding to top local maximas in the accumulator H

    Args:
        H: np.array, (num of rhos x num of thetas), hough transform accumulator
        rho_arr: np.array, dim=num of rhos, quantized rhos
        theta_arr: np.array, dim=num of thetas, quantized thetas
        num_lines: integer, number of lines we wish to detect in the image
        
    Returns:
        peak_rho_arr: np.array, dim=num_lines, top num_lines rhos by votes in the H
        peak_theta_arr: np.array, dim=num_lines, top num_lines thetas by votes in the H
    """
    
    # compute peak_rho_arr and peak_theta_arr
    # sort H using np.argsort, pick the top num_lines lines
    stride = len(theta_arr)
    sorted_flat = np.argsort(H, axis=None)[-num_lines:]
    peak_rho_arr = rho_arr[sorted_flat // stride]
    peak_theta_arr = theta_arr[sorted_flat % stride]

    assert(len(peak_rho_arr) == num_lines)
    assert(len(peak_theta_arr) == num_lines)
    return peak_rho_arr, peak_theta_arr


np.set_printoptions(suppress=True, precision=3)
image = np.zeros((7, 9))
image[3, :] = 255
image[:, 4] = 255
print(image)
image_m, image_o, image_x, image_y = edge_detection_nms(image, 0.1)
H, rho_arr, theta_arr = hough_transform(image_m, thres=700, rho_res=2, theta_res=18)
print(H)
peak_rho_arr, peak_theta_arr = peak_hough_lines(H, rho_arr, theta_arr, num_lines=4)
print("rho_arr", rho_arr)
print("theta_arr", theta_arr)
print("Top four (rho, then theta)")
print(peak_rho_arr)
print(peak_theta_arr)

# for image_idx in range(0, len(image_list)):
# image_idx = 0
# print(f"IMAGE {image_idx + 1} / {len(image_list)}")
# visualize(image_list[image_idx], constants)

[[  0.   0.   0.   0. 255.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 255.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 255.   0.   0.   0.   0.]
 [255. 255. 255. 255. 255. 255. 255. 255. 255.]
 [  0.   0.   0.   0. 255.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 255.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 255.   0.   0.   0.   0.]]
[[ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  3]
 [ 0  0  0  0  0  0  0  0  1  5  6]
 [ 0  0  0  0  0  0  0  1  6  6  5]
 [ 0  0  0  0  0  0  1  7  7  7  6]
 [ 4  1  0  0  1  2  8  6  6  5  4]
 [ 8  5  5  6  7 10  8  8  4  1  0]
 [ 6  7  7  7  8 10  7  2  0  0  0]
 [ 4  7  7  8  7  2  0  0  0  0  0]
 [ 2  4  5  3  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]]
rho_arr [-12. -10.  -8.  -6.  -4.  -2.   0.   2.   4.   6.   8.  10.  12.]
theta_arr [0.    0.314 0.628 0.942 1.257 1.571 1.885 2.199 2.513 2.827 3.142]
Top four (rho, then theta)
[2. 6. 4. 2.]


### Test Q3.1 and Q3.2

In [ ]:
### Uncomment for visualization
# image_idx = np.random.randint(0, len(image_list))
# visualize(image_list[image_idx], constants)

### Autograde Q3.1
5 points per test case, 3 test cases

### Autograde Q3.2
5 points per test case, 1 test case